# 集成学习介绍

- 集成学习：
    - 利用一些算法，把已经存在的机器学习算法结合在一起；
    - 特点：
        - 存在已知算法：预测，分类；
        - 结合已有算法；

- 集成的思想与算法具体实现
    - 数据集
        - 通过数据集把相同、或者不同的算法结合在一起（利用数据的差异）
    - 算法
        - 对于相同或者不相同的数据集，把不同的算法结合在一起。（利用算法的差异）

#  集成学习的总的思路

- 特别注意：
    - 集成学习不是基于算法（函数或者类）的结合，而是算法与数据形成的学习器（分类器或者回归器：对象【调用过fit函数，训练过的对象】】）

- 集成的主要目标：
    - 集成以后的学习器的效果，要好于单个学习器。
    - 被集成的学习器：基学习器（Base Learner：弱学习器Weaker）
    - 最后集成的学习器：强学习器
    - Stacking算法的：二次学习器：元（Meta  Learner）学习器；

- 集成的思路
    - Bootstrap Aggregating:自助聚合（Bagging）
        - 采用不同的数据集，使用相同的算法（可以使用不同算法），形成多个若学习器，然后采用结合策略，形成强学习器
    - Boosting 提升集成
        - 采用不同的数据集，使用相同的算法（可以使用不同算法），形成多个弱学习器。然后采用结合策略，形成强学习器
        
    - Stacking堆叠集成
        - 数据集相同，算法不同，形成多个弱学习器，然后采用结合策略，形成强学习器

- 集成的目标与手段：
    - boosting：减少残差（偏差）
        - AdaBoosting
        - GradientBoost：GBDT用于决策树的GradientBoost
    - Bagging：减少方差
        - 随机森林
    - Stacking：提升识别率，预测准确度（sklearn中好像没有提供）

- 结合策略
    - 平均法
        - 弱学习的结果值，直接平均（加权平均）作为最后的输出。
    - 投票法
        - 把某个样本的每个弱学习器的识别结果统计，分类最多的输出结果就是最终输出结果。

# sklearn中集成学习算法的应用

```python
    ensemble.AdaBoostClassifier([…])	An AdaBoost classifier.
    
    ensemble.BaggingClassifier([base_estimator, …])	A Bagging classifier.

    ensemble.ExtraTreesClassifier([…])	An extra-trees classifier.

    ensemble.GradientBoostingClassifier([loss, …])	Gradient Boosting for classification.

    ensemble.IsolationForest([n_estimators, …])	Isolation Forest Algorithm
    
    ensemble.RandomForestClassifier([…])	A random forest classifier.
    
    ensemble.VotingClassifier(estimators[, …])	Soft Voting/Majority Rule classifier for unfitted estimators.
```

In [19]:
import numpy as np
import sklearn.datasets as ds
import sklearn.ensemble as en
import sklearn.model_selection as ms
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# 实现一个训练与测试方法（数据集加载，数据集切分）
data, target = ds.load_iris(return_X_y=True)
data_train, data_test, target_train, target_test = ms.train_test_split(data, target, test_size=0.2)

# 创建集成强学习器
classifier_bagging = en.BaggingClassifier(n_estimators=100)
classifier_extra = en.ExtraTreesClassifier(n_estimators=100)
classifier_rf =  en.RandomForestClassifier(n_estimators=100)
classifier_ada =  en.AdaBoostClassifier(n_estimators=100)
classifier_gradient =  en.GradientBoostingClassifier(n_estimators=100)

# 投票学习器
# 线性回顾
# 逻辑回归
c1 = LogisticRegression(solver='lbfgs',multi_class='auto')
# 支撑向量机
c2 = SVC(C=1.0, kernel='rbf',gamma='scale')
# 集成算法（KNN）


classifier_vote =  en.VotingClassifier(estimators=[('c1', c1),('c2',c2 )])   # 需要多个学习器


def  test_classifier(classifier):
    # 使用训练与测试方案，观察训练与测试效果。
    total_num = len(target_test)
    # 训练
    classifier.fit(data_train, target_train)
    # 测试
    pre = classifier.predict(data_test)
    # 统计打印数据
    correct_num = (pre ==target_test).sum()
    # 打印
    print(F'识别正确数：{correct_num}，识别正确率：{100.0*correct_num/total_num:6.2f}%')

test_classifier(classifier_bagging)
test_classifier(classifier_extra)
test_classifier(classifier_rf)
test_classifier(classifier_ada)
test_classifier(classifier_gradient)
test_classifier(classifier_vote) 

识别正确数：28，识别正确率： 93.33%
识别正确数：28，识别正确率： 93.33%
识别正确数：28，识别正确率： 93.33%
识别正确数：26，识别正确率： 86.67%
识别正确数：28，识别正确率： 93.33%
识别正确数：29，识别正确率： 96.67%


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


# 集成学习的算法

## Bagging集成算法

### 算法的过程
- Bootstrap Aggregating
    - 通过随机抽样，形成多个数据集。（抽样方法有很多种，其中存在一种抽样方式：Bootstrap抽样）
    - 每个数据集训练出一个弱学习器。
    - 对测试样本，，每个若学习器得到一个预测效果，
    - 对多个预测效果进行投票。作为最后强学习器的预测结果。

- 只有一个，如果应用到决策树：
    - 随机森林

### 算法的关键

- Bootstrap随机抽样法的作用
    - 解决了小样本数据集的问题。
    
- Bootstrap随机抽样方法：
    - 假设样本$X = \{ x_1, x_2,\cdots, x_n \}$，$n$是样本容量。
    - 抽样的数量也是$n$个。
    - 抽样的方式是：有放回抽样
        - 抽取的n个样本的训练集其中存在重复样本。
- Bootstrap随机抽样的数学依据：
    - 对鸢尾花数据集抽样m次，得到Bootstrap抽样数据集m个。
    - 训练的结果：结构正确的置信度可以通过bootsrap抽样计算出来：Bagging算法投票的预测结果，置信度是95%。
    
    - m个Bootstrap数据集训练出来的基学习器的预测结果是m个，对m结果做频次统计，预测正确的可信度95%（是0的预测为0，1的预测为1），

- 改算法固定在决策树上
    - 随机森林。

In [ ]:
def  bootstrap_sample(data , target):
    data_sample = []
    target_sample = []
    # 得到data的长度n
    # 循环n次
    # 取0~n-1之间的均匀随机整数。
    # 使用整数作为下标，得到采样数据
    
    # 返回所有的采样数据
    return data_sample, target_sample

In [ ]:
# 如果算法使用决策树，就是随机森林。
class Bagging:
    
    base_learners = []
    base_num =20
    def fit(data_train, target_train):
        # 根据base_num基学习器的个数，调用bootstrap采样，得到base_num个数据集
        # 使用指定的学习器类，循环base_num次构造多个学习器base_learners。
        # 循环使用不同的数据集，训练base_learners中的对象
        pass
    
    
    def predict(data_test):
        # 使用上面训练出来的base_num个学习器，预测data_test。
        # 得到base_num个结果
        # 对base_num个结果投票，作为最后的返回值。
        pass


## Boosting集成算法

- 与Bag一样都是在数据集做文章，尽量采用相同的算法：
    - 直接原始数据集加权；得到新的数据集；得到新的结果。
    
    - 最终得到一个强学习器
        - 采用误差损失函数最小。

### AdaBoosting算法
    - 加权计算方式1

#### 预测模型

- 弱学习器怎么结合成强学习器
    - 通过m次变换权重，得到m个数据集$X_i$
    - 训练出m个基学习器$G_i(x)$
    - 采用平均的值对方式得到最终的结果。
    - $f(x) =\alpha_1 G_1(x) + \alpha_2 G_2(x) + \cdots +\alpha_m G_m(x)$
    - $f(x) = \sum \limits _{i=1} ^{m} \alpha_i G_i(x)$

#### 损失模函数

- $L(y, f(x)) = \sum \limits_{i=1} ^ {n} L(y_i, f(x_i))$

- 一般传统的度量方式：$L(y_i, f(x_i)) =\sqrt{ {(y_i - f(x_i))}^2}$

- AdaBoosting的误差度量：采用是指数函数

    - $L(y,  f(x)) = e^{-yf(x)}$

- $L(x, f(x)) = \sum \limits_{i=1} ^{n} e^{y_i \sum \limits _{k=1}^{m} \alpha_k G_k(x_i)}$

####  Ada损失模型的简化模型

- 多元简化成一元

- 简化预测模型：
    - $G_1(x)$：认为所有样本的重要性一样，取权重$\dfrac{1}{n}$,每个样本乘以权重，得到新的样本集$\bar{X}$
    - $f_1= \alpha_1 G_1(x)$

- $f_k (x) = f_{k-1}(x) + \alpha_k G_k(x) $

- 损失模型简化：
    - $L(\alpha_k, G_k(x), w_k) = \sum \limits _{i=1} ^{n} e ^{y_i f_k(x_i)} =  \sum \limits _{i=1} ^{n} e ^{y_i (f_{k-1} (x_i) + \alpha_k G_k(x_i))}$

#### AdaBoosting算法过程
- 下面算法过程中的公式全部来自上面的损失函数最小值推导的公式。

1. 假设：
     - 样本集$X = \{ (x_1, y_1), (x_2, y_2), \cdots, (x_n, y_n) \}$\
     - $x_i \in R$
     - $y_i \in \{-1, 1\}$, 把问题假设二分类。

2. 初始化：
    - 初始化一个权重：$(w_1, w_2, \cdots, w_n)$, $w_i= \dfrac{1}{n}$

3. 得到训练样本集
    - $X_1 = \{(w_1 x_1, y_1), \cdots, (w_n x_n, y_n)\}$

4. 使用指定的算法与数据集$X_1$训练得到基学习器$G_1(x) : x \to \{-1, 1\}$

5. 计算错误率
     - $e_1 = \sum _{i=1} ^{n} w_i I(G_1(x_i))$
         - 其中$I$函数的定义:$G(x_i)$是错误，$I$返回1，否则返回0
         - $\sum \limits _ {i=1} ^n w_i   =1$

6. 计算$\alpha_1$


- $ \alpha_1 = \dfrac{1}{2} ln (\dfrac{1 - e_1}{e_1} )$

7. 计算下一次的权重系数$(w_1 ^{(2)}, \cdots,w_n ^{(2)})$

- 权重系数的计算步骤：
    1. 计算规范化因子：
        - $z_1 = \sum \limits _{i=1} ^ n w_i^{(1)} e ^{ - \alpha_1 y_i G_1(x_i)}$
        
    2. $w_i^{(2)} = \dfrac{w_i^{(1)} e ^{ \alpha_1 y_i G_1(x_1)} }{z_1} $

- 注意：
    - $\sum \limits _{i=1} ^{n} w_i^{(2)} = 1$

8. 使用$w_i^{(2)}$得到新的数据集，训练$G_2(x)$，然后开始循环到m次结束，

### GradientBoosting算法
    - 加权计算方式2

## Stacking集成算法
    - 在算法上做文章。